In [1]:
import netCDF4 as nc
from mpl_toolkits.basemap import Basemap,cm
from matplotlib.colors import LogNorm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as pl
import numpy as np
import math

In [2]:
maindir = '/disk02/UNCERTAINTIES/Monte-Carlo/'
datadir = 'SWF/DATA/L2_L3/CentralPacific/Standard/'
fname = 'S1999229224418.L3m'
fpath = maindir + datadir + fname
bands = ['412','443','490','510','555','670','765','865']
rrsDict = dict.fromkeys(bands[:-2])
rrsUncDict = dict.fromkeys(bands[:-2])
dss = nc.Dataset(fpath)

In [4]:
rrsPercUncDict = dict.fromkeys(bands[:-2])
dvars = dss.variables
maxLat,minLat = dss.northernmost_latitude,dss.southernmost_latitude
maxLon,minLon = dss.easternmost_longitude,dss.westernmost_longitude
lat = dvars['lat'][:]
lon = dvars['lon'][:]
timeRange=dss.time_coverage_start,dss.time_coverage_end
for band in bands[:-2]:
    rrsDict[band] = dvars['Rrs_' + band][:]
    rrsUncDict[band] = dvars['Rrs_unc_' + band][:]
    rrsPercUncDict[band] = rrsUncDict[band] / rrsDict[band] * 100
dss.close()

In [5]:
rpu412=rrsPercUncDict['412']

In [76]:
lons,lats=np.meshgrid(lon,lat)

In [77]:
lats.shape,lons.shape,rpu412.shape

((1556, 4318), (1556, 4318), (1556, 4318))

In [21]:
lonm=np.ma.array(lons,mask=rpu412.mask)
latm=np.ma.array(lats,mask=rpu412.mask)

In [78]:
Mlon=np.ma.masked_outside(lons,-180,-120)

In [79]:
newRpu412=np.ma.array(rpu412.data,mask=rpu412.mask+Mlon.mask)

In [80]:
newlats=np.ma.array(lats,mask=newRpu412.mask)
newlons=np.ma.array(lons,mask=newRpu412.mask)

In [84]:
minLat,maxLat=newlats.compressed().min(),newlats.compressed().max()

In [86]:
minLon,maxLon=newlons.compressed().min(),newlons.compressed().max()

In [103]:
ms = Basemap(projection='cyl',lon_0=-150,llcrnrlon=minLon,llcrnrlat=minLat,
             urcrnrlat=maxLat,urcrnrlon=maxLon)
ms.drawcoastlines()
ms.fillcontinents();
ms.drawmapboundary(fill_color='k')

In [88]:
colorGistEart=[(0.33,0.33,0.33)] + [(pl.cm.gist_earth(i)) for i in range(1,256)]
new_map = LinearSegmentedColormap.from_list('new_map',colorGistEart,N=256)
imgRpu412 = ms.imshow(np.flipud(newRpu412),norm=LogNorm(vmin=0.1,vmax=1000.0),
              cmap=new_map,interpolation='bilinear')
cbRpu412 = ms.colorbar(imgRpu412,'bottom',size="5%",pad='4%')

In [ ]:
scatRpu412=ms.scatter(newlons.compressed(),)

In [89]:
imgRpu412.figure.savefig('pImgTest.png')

In [54]:
imgRpu412.figure.clf()

NameError: name 'imgRpu412' is not defined

In [53]:
del imgRpu412
del ms

In [104]:
pc=ms.pcolor(newlons,newlats,newRpu412,latlon=True)
#cbpc = ms.colorbar(pc,'bottom',size="5%",pad='4%')
pc.figure.set_tight_layout(True)

In [105]:
pc.figure.savefig('pctest.png')

In [100]:
pc.figure.clf()